In [1]:
!pip install googlemaps networkx
!pip install osmnx networkx geopy openrouteservice
!pip install osmnx networkx geopy openrouteservice folium
!pip install flask-ngrok
!pip install pyngrok

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40715 sha256=a59b14e57679396f60c4c3c10997969b4e2d30da4caa669dabe501dadfbf7a0c
  Stored in directory: /root/.cache/pip/wheels/f1/09/77/3cc2f5659cbc62341b30f806aca2b25e6a26c351daa5b1f49a
Successfully built googlemaps
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 kB 5.3 MB/s eta 0:00:00


In [2]:
import osmnx as ox
import networkx as nx
import openrouteservice
from geopy.distance import geodesic

# 📌 OpenRouteService API Setup (Free Alternative to Google Maps)
API_KEY = "5b3ce3597851110001cf62483e5592a6194d42739dc544c5b850fbf1"  # Replace with your free API key
ors_client = openrouteservice.Client(key=API_KEY)

# 📌 Define Vehicles (Capacity, Speed, Location)
vehicles = [
    {"id": 1, "location": (28.7041, 77.1025), "capacity": 100, "speed": 40},  # Delhi
    {"id": 2, "location": (19.0760, 72.8777), "capacity": 80, "speed": 50},   # Mumbai
]

# 📌 Define Orders (Pickup & Drop, Weight, Priority)
orders = [
    {"id": 101, "pickup": (28.6139, 77.2090), "dropoff": (28.5355, 77.3910), "weight": 30, "priority": 1},
    {"id": 102, "pickup": (19.2183, 72.9781), "dropoff": (19.0760, 72.8777), "weight": 50, "priority": 2},
]

# 📌 Function to get shortest travel time using OpenRouteService
def get_travel_time(origin, destination):
    try:
        route = ors_client.directions(
            coordinates=[(origin[1], origin[0]), (destination[1], destination[0])],
            profile="driving-car",
            format="json"
        )
        duration = route["routes"][0]["summary"]["duration"] / 60  # Convert to minutes
        return duration
    except Exception as e:
        print(f"⚠️ Error getting route: {e}")
        return float("inf")

# 📌 Assign Orders to Vehicles Dynamically
def assign_orders(vehicles, orders):
    assigned_orders = []

    for order in orders:
        best_vehicle = None
        min_cost = float("inf")

        for vehicle in vehicles:
            if vehicle["capacity"] >= order["weight"]:
                travel_time = get_travel_time(vehicle["location"], order["pickup"])
                total_cost = travel_time + (order["priority"] * 10)

                if total_cost < min_cost:
                    min_cost = total_cost
                    best_vehicle = vehicle

        if best_vehicle:
            assigned_orders.append({"vehicle": best_vehicle["id"], "order": order["id"], "cost": min_cost})
            best_vehicle["location"] = order["dropoff"]  # Update vehicle location

    return assigned_orders

# 📌 Run the Algorithm
optimal_routes = assign_orders(vehicles, orders)

# 📌 Print the Assigned Orders
print("\n✅ Optimal Order Assignments:")
for assignment in optimal_routes:
    print(f"🚚 Vehicle {assignment['vehicle']} assigned to Order {assignment['order']} with cost: {assignment['cost']} mins")



✅ Optimal Order Assignments:
🚚 Vehicle 1 assigned to Order 101 with cost: 33.43 mins
🚚 Vehicle 2 assigned to Order 102 with cost: 36.403333333333336 mins


In [3]:
import osmnx as ox
import networkx as nx
import openrouteservice
import folium
from geopy.distance import geodesic

# 📌 OpenRouteService API Setup (Free Alternative to Google Maps)
API_KEY = "5b3ce3597851110001cf62483e5592a6194d42739dc544c5b850fbf1"  # 🔴 Replace with your OpenRouteService API Key
ors_client = openrouteservice.Client(key=API_KEY)

# 📌 Define Vehicles (Location, Capacity, Speed)
vehicles = [
    {"id": 1, "location": (28.7041, 77.1025), "capacity": 100, "speed": 40},  # Delhi
    {"id": 2, "location": (19.0760, 72.8777), "capacity": 80, "speed": 50},   # Mumbai
    {"id": 3, "location": (13.0827, 80.2707), "capacity": 120, "speed": 45},  # Chennai
    {"id": 4, "location": (22.5726, 88.3639), "capacity": 90, "speed": 42},   # Kolkata
]

# 📌 Define Orders (Pickup, Drop-off, Weight, Priority)
orders = [
    {"id": 101, "pickup": (28.6139, 77.2090), "dropoff": (28.5355, 77.3910), "weight": 30, "priority": 1},  # Delhi
    {"id": 102, "pickup": (19.2183, 72.9781), "dropoff": (19.0760, 72.8777), "weight": 50, "priority": 2},  # Mumbai
    {"id": 103, "pickup": (13.0051, 80.2526), "dropoff": (13.0827, 80.2707), "weight": 40, "priority": 1},  # Chennai
    {"id": 104, "pickup": (22.5958, 88.2636), "dropoff": (22.5726, 88.3639), "weight": 70, "priority": 3},  # Kolkata
    {"id": 105, "pickup": (28.4595, 77.0266), "dropoff": (28.7041, 77.1025), "weight": 60, "priority": 2},  # Gurgaon
]

# 📌 Function to Get Travel Time Using OpenRouteService with Fallback
def get_travel_time(origin, destination, speed):
    try:
        route = ors_client.directions(
            coordinates=[(origin[1], origin[0]), (destination[1], destination[0])],
            profile="driving-car",
            format="json"
        )
        return route["routes"][0]["summary"]["duration"] / 60  # Convert seconds to minutes
    except Exception as e:
        print(f"⚠️ API Error: {e}. Using estimated distance method.")
        # Fallback: Estimate time using geodesic distance and speed
        distance_km = geodesic(origin, destination).km
        return (distance_km / speed) * 60  # Convert hours to minutes

# 📌 Assign Orders to Vehicles Dynamically
def assign_orders(vehicles, orders):
    assigned_orders = []
    routes = []

    for order in orders:
        best_vehicle = None
        min_cost = float("inf")

        for vehicle in vehicles:
            if vehicle["capacity"] >= order["weight"]:
                travel_time = get_travel_time(vehicle["location"], order["pickup"], vehicle["speed"])
                total_cost = travel_time + (order["priority"] * 10)

                if total_cost < min_cost:
                    min_cost = total_cost
                    best_vehicle = vehicle

        if best_vehicle:
            assigned_orders.append({"vehicle": best_vehicle["id"], "order": order["id"], "cost": min_cost})
            routes.append((best_vehicle["location"], order["pickup"], order["dropoff"]))
            best_vehicle["location"] = order["dropoff"]  # Update vehicle location

    return assigned_orders, routes

# 📌 Run the Algorithm
optimal_routes, route_paths = assign_orders(vehicles, orders)

# 📌 Print the Assigned Orders
print("\n✅ Optimal Order Assignments:")
for assignment in optimal_routes:
    print(f"🚚 Vehicle {assignment['vehicle']} assigned to Order {assignment['order']} with cost: {assignment['cost']:.2f} mins")

# 📌 Visualization: Plot Routes on a Map
map_center = (20.5937, 78.9629)  # Center of India
m = folium.Map(location=map_center, zoom_start=5)

# 📌 Plot Vehicles
for vehicle in vehicles:
    folium.Marker(
        location=vehicle["location"],
        popup=f"🚚 Vehicle {vehicle['id']}",
        icon=folium.Icon(color="blue", icon="truck", prefix="fa"),
    ).add_to(m)

# 📌 Plot Orders
for order in orders:
    folium.Marker(
        location=order["pickup"],
        popup=f"📦 Order {order['id']} (Pickup)",
        icon=folium.Icon(color="green", icon="flag", prefix="fa"),
    ).add_to(m)
    folium.Marker(
        location=order["dropoff"],
        popup=f"📦 Order {order['id']} (Dropoff)",
        icon=folium.Icon(color="red", icon="flag", prefix="fa"),
    ).add_to(m)

# 📌 Draw Routes with Error Handling
for start, pickup, dropoff in route_paths:
    try:
        route = ors_client.directions(
            coordinates=[(start[1], start[0]), (pickup[1], pickup[0]), (dropoff[1], dropoff[0])],
            profile="driving-car",
            format="geojson"
        )

        if "routes" in route and route["routes"]:
            folium.PolyLine(
                locations=[(coord[1], coord[0]) for coord in route["routes"][0]["geometry"]["coordinates"]],
                color="blue",
                weight=3,
                opacity=0.8
            ).add_to(m)
        else:
            print(f"⚠️ No valid route found for vehicle at {start}. Skipping...")
    except Exception as e:
        print(f"⚠️ Error drawing route: {e}")


# 📌 Save and Show Map
map_filename = "vehicle_routing_map.html"
m.save(map_filename)
print(f"\n🌍 Map saved as '{map_filename}'. Open the file in a browser to view it.")



✅ Optimal Order Assignments:
🚚 Vehicle 1 assigned to Order 101 with cost: 33.43 mins
🚚 Vehicle 2 assigned to Order 102 with cost: 36.40 mins
🚚 Vehicle 3 assigned to Order 103 with cost: 26.76 mins
🚚 Vehicle 4 assigned to Order 104 with cost: 48.48 mins
🚚 Vehicle 1 assigned to Order 105 with cost: 73.77 mins
⚠️ No valid route found for vehicle at (28.7041, 77.1025). Skipping...
⚠️ No valid route found for vehicle at (19.076, 72.8777). Skipping...
⚠️ No valid route found for vehicle at (13.0827, 80.2707). Skipping...
⚠️ No valid route found for vehicle at (22.5726, 88.3639). Skipping...
⚠️ No valid route found for vehicle at (28.5355, 77.391). Skipping...

🌍 Map saved as 'vehicle_routing_map.html'. Open the file in a browser to view it.


In [4]:
import requests
import folium
import json
from flask import Flask, request, jsonify
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from sklearn.linear_model import LinearRegression
from flask_ngrok import run_with_ngrok  # Required for Google Colab

# Flask App Initialization
app = Flask(__name__)
run_with_ngrok(app)  # Enables public URL in Google Colab

# OpenRouteService API Key
ORS_API_KEY = "5b3ce3597851110001cf62483e5592a6194d42739dc544c5b850fbf1"
ORS_BASE_URL = "https://api.openrouteservice.org/v2/directions/driving-car"

# Vehicle Data
vehicles = [
    {"id": 1, "location": [12.9716, 77.5946], "capacity": 100, "fuel_efficiency": 15},  # Example vehicle
]

# Orders (Dynamically Updated)
orders = []

# ETA Prediction Model (Machine Learning)
def train_eta_model():
    X_train = np.array([[1], [2], [3], [4], [5]])  # Distance in km
    y_train = np.array([10, 20, 30, 40, 50])  # ETA in minutes
    model = LinearRegression().fit(X_train, y_train)
    return model

eta_model = train_eta_model()

def get_eta_prediction(distance):
    return eta_model.predict(np.array([[distance]]))[0]

# Function to Fetch Optimized Route (Parallel Processing)
def get_optimized_route(start, end):
    try:
        url = f"{ORS_BASE_URL}?api_key={ORS_API_KEY}&start={start[1]},{start[0]}&end={end[1]},{end[0]}"
        response = requests.get(url, timeout=5)  # 5s timeout for quick response
        if response.status_code == 200:
            route_data = response.json()
            distance = route_data["routes"][0]["summary"]["distance"] / 1000  # Convert meters to km
            duration = route_data["routes"][0]["summary"]["duration"] / 60  # Convert seconds to minutes
            return route_data, distance, duration
    except Exception as e:
        print(f"Error fetching route: {e}")
    return None, None, None

# Order Assignment with Dynamic Route Optimization
@app.route("/assign_order", methods=["POST"])
def assign_order():
    data = request.json
    pickup = data["pickup"]
    dropoff = data["dropoff"]
    weight = data.get("weight", 10)

    best_vehicle = None
    best_cost = float("inf")

    with ThreadPoolExecutor() as executor:  # Parallel Processing for faster results
        futures = {executor.submit(get_optimized_route, v["location"], pickup): v for v in vehicles if v["capacity"] >= weight}
        for future in futures:
            vehicle = futures[future]
            _, distance, _ = future.result()
            if distance is not None:
                fuel_cost = distance / vehicle["fuel_efficiency"]
                if fuel_cost < best_cost:
                    best_cost = fuel_cost
                    best_vehicle = vehicle

    if best_vehicle:
        best_vehicle["location"] = dropoff
        orders.append({"vehicle_id": best_vehicle["id"], "pickup": pickup, "dropoff": dropoff})
        return jsonify({"message": "Order assigned", "vehicle": best_vehicle}), 200
    else:
        return jsonify({"error": "No suitable vehicle found"}), 400

# Route Visualization (Generates Map)
@app.route("/view_map", methods=["GET"])
def visualize_routes():
    map_ = folium.Map(location=[12.9716, 77.5946], zoom_start=12)
    for order in orders:
        folium.Marker(order["pickup"], icon=folium.Icon(color="green"), popup="Pickup").add_to(map_)
        folium.Marker(order["dropoff"], icon=folium.Icon(color="red"), popup="Dropoff").add_to(map_)
    map_.save("delivery_routes.html")
    return jsonify({"message": "Map saved as delivery_routes.html"}), 200

# Run Flask App
if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            